In [1]:
# from joblib import Parallel, delayed 
# import joblib 

# # Load the model from the file 
# RandomForest_from_joblib = joblib.load('Random_Forest2.pkl') 
 
# RandomForest_from_joblib

In [2]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from colorama import Fore
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from tld import get_tld, is_tld
from keras.layers import Dense, LSTM
from keras.models import Sequential
from keras.layers import Dense, LSTM

In [3]:
from keras.models import load_model

In [4]:
model = load_model("model_lstm_fixed_151.h5")

In [5]:
data = pd.read_csv("test_x.csv")
data.head()

,url
0,18.154.7.62


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     1 non-null      object
dtypes: object(1)
memory usage: 136.0+ bytes


In [7]:
data.isnull().sum()

url    0
dtype: int64

In [8]:
#replace www
data['url'] = data['url'].replace('www.', '', regex=True)
data

,url
0,18.154.7.62


In [9]:
data.head()

,url
0,18.154.7.62


In [10]:
data['url_len'] = data['url'].apply(lambda x: len(str(x)))

In [11]:
def process_tld(url):
    try:
        res = get_tld(url, as_object = True, fail_silently=False,fix_protocol=True)
        domain= res.parsed_url.netloc
    except :
        domain= None
    return domain

In [12]:
data['domain'] = data['url'].apply(lambda i: process_tld(i))

In [13]:
data.head()

,url,url_len,domain
0,18.154.7.62,11,None


In [14]:
feature = ['@','?','-','=','.','#','%','+','$','!','*',',','//']
for a in feature:
    data[a] = data['url'].apply(lambda i: i.count(a))

In [15]:
data.head()

,url,url_len,domain,@,?,-,=,.,#,%,+,$,!,*,",",//
0,18.154.7.62,11,None,0,0,0,0,3,0,0,0,0,0,0,0,0


In [16]:
def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 0
    else:
        # print 'No matching pattern found'
        return 1

In [17]:
data['abnormal_url'] = data['url'].apply(lambda i: abnormal_url(i))

In [18]:
def httpSecure(url):
    htp = urlparse(url).scheme
    match = str(htp)
    if match=='https':
        # print match.group()
        return 0
    else:
        # print 'No matching pattern found'
        return 1

In [19]:
data['https'] = data['url'].apply(lambda i: httpSecure(i))

In [20]:
def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits

In [21]:
data['digits']= data['url'].apply(lambda i: digit_count(i))

In [22]:
def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters

In [23]:
data['letters']= data['url'].apply(lambda i: letter_count(i))

In [24]:
def Shortining_Service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0

In [25]:
data['Shortining_Service'] = data['url'].apply(lambda x: Shortining_Service(x))

In [26]:
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4 with port
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}|'
        '([0-9]+(?:\.[0-9]+){3}:[0-9]+)|'
        '((?:(?:\d|[01]?\d\d|2[0-4]\d|25[0-5])\.){3}(?:25[0-5]|2[0-4]\d|[01]?\d\d|\d)(?:\/\d{1,2})?)', url)  # Ipv6
    if match:
        return 1
    else:
        return 0

In [27]:
data['having_ip_address'] = data['url'].apply(lambda i: having_ip_address(i))

In [28]:
data['having_ip_address'].value_counts()

1    1
Name: having_ip_address, dtype: int64

In [29]:
X = data.drop(['url','domain'],axis=1)#,'type_code'


In [30]:
X

,url_len,@,?,-,=,.,#,%,+,$,!,*,",",//,abnormal_url,https,digits,letters,Shortining_Service,having_ip_address
0,11,0,0,0,0,3,0,0,0,0,0,0,0,0,1,1,8,0,0,1


In [31]:
X.shape

(1, 20)

In [32]:
test = model.predict(X)

1/1 [==============================] - 0s 364ms/step


In [33]:
if test < 0.8:
    print("safe")
else :
   print("phishing")



safe
